In [1]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen
import glob
import altair as alt
import time
import itertools
import numpy as np
import operator

pd.options.display.max_columns = 100



In [3]:
# Helper Functions

def Filter(string, substr): 
        return [str for str in string if
                any(sub in str for sub in substr)] 
    
def NOTFilter(string, substr): 
    return [str for str in string if
            any(sub not in str for sub in substr)]

In [5]:
def fbref_league_history(league_id = [9,11,12,13,20], first_season = 2010):
    history = []
    for i in league_id:
        comp_history_url = "https://fbref.com/en/comps/" + str(i) + "/history" 
        #print(comp_history_url)

        r=requests.get(comp_history_url)
        soup=BeautifulSoup(r.content, "html.parser")

        find_seasons = soup.find_all(class_ = "left")

        all_seasons_url = []
        for k in range(0, len(find_seasons)):
            if find_seasons[k].get('data-stat') == "season":
                temp = "https://fbref.com" + find_seasons[k].find_all("a")[0].attrs["href"]
                all_seasons_url.append(temp)

        history.append(all_seasons_url)
        time.sleep(0.1)

    # All histories in one array
    history  = list(itertools.chain(*history))

    seasons = list(map(lambda x: str(x)+"-"+str(x+1), np.arange(1950, first_season, 1)))
    for i in seasons:
        history = NOTFilter(history, [i])
    del seasons

    return history


In [6]:
# Getting history for each league (England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20)
history_england = fbref_league_history(league_id = [9])

In [7]:
history_italy = fbref_league_history(league_id = [11])

In [8]:
history_spain = fbref_league_history(league_id = [12])

In [9]:
history_france = fbref_league_history(league_id = [13])

In [10]:
history_germany = fbref_league_history(league_id = [20])

In [12]:
def fbref_team_url_history(league_history):
    team_season_url = []
    for league_season_url in league_history:
        r=requests.get(league_season_url)
        soup=BeautifulSoup(r.content, "html.parser")
        teams = soup.find("table").find_all("a")
        teams = list(map(lambda x: "https://fbref.com" + x["href"], teams))
        teams = Filter(teams, ["/en/squads/"])
        team_season_url.append(teams)

    # All histories in one array
    team_season_url  = list(itertools.chain(*team_season_url))
    return team_season_url



In [20]:
# Premier League (England) Seasons (England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20)
team_season_url_england = fbref_team_url_history(history_england)

In [21]:
# Serie A (Italy) Seasons (England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20)
team_season_url_italy = fbref_team_url_history(history_italy)

In [22]:
# La Liga (Spain) Seasons (England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20)
team_season_url_spain = fbref_team_url_history(history_spain)

In [23]:
# Ligue 1 (France) Seasons (England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20)
team_season_url_france = fbref_team_url_history(history_france)

In [24]:
# Bundesliga (Germany) Seasons (England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20)
team_season_url_germany = fbref_team_url_history(history_germany)

In [26]:
def fbref_player_url(team_season_url):
    player_url = []
    for turl in team_season_url:
        r=requests.get(turl)
        soup=BeautifulSoup(r.content, "html.parser")
        soup.find("div", {"id":"all_stats_standard"})
        players = soup.find("tbody").find_all("a")
        players = list(map(lambda x: x["href"], players))
        players = Filter(players, ["/en/players/"])
        players = NOTFilter(players, ["matchlogs"])
        player_url.append(list(map(lambda x: "https://fbref.com" + x, players)))
        time.sleep(0.01)
    player_url  = list(itertools.chain(*player_url))
    return player_url

In [27]:
# Getting players urls for England (England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20)
player_url_england = fbref_player_url(team_season_url_england)

In [28]:
# Getting players urls for Italy (England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20)
player_url_italy = fbref_player_url(team_season_url_italy)

KeyboardInterrupt: 

In [ ]:
# Getting players urls for Spain (England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20)
player_url_spain = fbref_player_url(team_season_url_spain)

In [ ]:
# Getting players urls for France (England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20)
player_url_france = fbref_player_url(team_season_url_france)

In [ ]:
# Getting players urls for Germany (England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20)
player_url_germany = fbref_player_url(team_season_url_germany)

In [34]:
df = pd.DataFrame(player_url_england)
df[0].value_counts()

https://fbref.com/en/players/1ecb65be/Theo-Walcott           14
https://fbref.com/en/players/01226327/Ryan-Bertrand          13
https://fbref.com/en/players/86dd77d1/Kyle-Walker            13
https://fbref.com/en/players/2f90f6b8/James-Milner           13
https://fbref.com/en/players/2ff964a0/Aaron-Lennon           13
                                                             ..
https://fbref.com/en/players/b3576b7c/Erwin-Mulder            1
https://fbref.com/en/players/ed2b33cb/Yuri-Zhirkov            1
https://fbref.com/en/players/9fc0f253/Matt-Miazga             1
https://fbref.com/en/players/05380aa0/Richard-Kingson         1
https://fbref.com/en/players/0f933743/Francisco-Sierralta     1
Name: 0, Length: 2450, dtype: int64

In [39]:
len(player_url_england)

7179

In [43]:
def fbref_player_info(player_url):
    player_info = []
    for completed, i in enumerate(player_url):

        # PlayerId
        playerId = i.replace("https://fbref.com/en/players/", "").split("/")[0]

        # Request
        r=requests.get(i)
        soup=BeautifulSoup(r.content, "html.parser")

        # Meta
        meta = soup.find("div", {"id":"meta"})

        # Player Name
        playerName = soup.find("h1").find("span").get_text()
        
        # Nationality
        birthplace = meta.find("span", {"itemprop": "birthPlace"}).text.replace("\n", "").strip().split(", ")
        nationality = birthplace[len(birthplace)-1]
        

        # Player Photos
        try:
            photo = soup.find("div", {"class":"media-item"}).find("img").attrs["src"]
        except:
            photo = np.nan


        # Birth
        try:
            birth = meta.find("span", {"itemprop": "birthDate"}).text.replace("\n", "").strip()
            #soup.find("div", {"id":"meta"}).find("span", {"id":"necro-birth"})['data-birth']
        except:
            birth = np.nan

        # Height
        try:
            height = meta.find("span", {"itemprop":"height"}).text.replace("cm", "")
        except:
            height = np.nan

        # Weight
        try:
            weight = soup.find("div", {"id":"meta"}).find("span", {"itemprop":"weight"}).text.replace("kg", "")
        except :
            weight = np.nan


        detail = meta.find_all("p")

        # Player Full Name
        if len(Filter([detail[0].text], ["Position", "Club", "Born", "National Team", "Citizenship"])) > 0:
                playerFullName = np.nan
        else:
            playerFullName = detail[0].get_text()

        # Position & Footed
        fp = list(map(lambda x: str(x), detail))
        position = Filter(fp, ["Position"])
        footed = Filter(fp, ["Footed"])
        if len(position) > 0:
            position = position[0].split("<strong>")[1].replace("Position:</strong>","").replace("\n", "").replace("<p>", "").replace("</p>", "").replace("\xa0", "").replace("▪", "").split("<span")[0].strip()
        else:
            position = np.nan

        if len(footed) > 0:
            footed = footed[0].split("<strong>Footed:</strong>")[1].split("<span")[0].strip().replace("</p>", "").upper()
            footed = footed.split("% ")
            if len(footed) > 1:
                foot = footed[1]
                foot_ability = int(footed[0]) 
            else:
                foot = footed[0]
                foot_ability = 100
        else:
            foot = np.nan
            foot_ability = np.nan

        # International Reputation
        try:
            ir = soup.find("ul", {"id":"bling"}).find_all("a")
            ir = list(map(lambda x: x.text.strip(), ir))
            ir = '||'.join(map(str, ir))  # While the variable will be made || should be separated with
        except:
            ir = np.nan
            
        #Social Media
        sm = Filter(list(map(lambda x: x["href"], meta.find_all("a", href = True))), ["twitter", "instagram"])
        try:
            tw = Filter(sm, ["twitter"])[0]
        except:
            tw = np.nan
        try:
            ins = Filter(sm, ["instagram"])[0]
        except:
            ins = np.nan

        # Data Frame
        temp = pd.DataFrame({
            "FBRefId":[playerId],
            "PlayerName":[playerName],
            "PlayerFullName":[playerFullName],
            "Nationality":[nationality],
            "Photo":[photo],
            "Birth":[birth],
            "Height":[height],
            "Weight":[weight],
            "Position":[position],
            "Foot":[foot],
            "FootAbility":[foot_ability],
            "InternationalReputation":[ir],
            "PlayerUrl":[i],
            "Twitter":[tw],
            "Instagram":[ins]
        })    

        temp["PlayerFullName"] = np.where(temp.PlayerFullName.isnull(), temp.PlayerName, temp.PlayerFullName)

        player_info.append(temp)

        # Print Message
        sys.stdout.write("\r{0} players have just scraped from FBRef!".format(completed+1))
        sys.stdout.flush()

        # System Sleep
        time.sleep(0.01) 

    # Write Player Info
    player_info = pd.concat(player_info)
    
    return player_info



In [44]:
# Getting players info for England (England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20)
player_info_england = fbref_player_info(player_url_england)

264 players have just scraped from FBRef!

In [ ]:
# Saving Player Info for all countries

pd.DataFrame(player_info_england).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_england.csv', index=False)
pd.DataFrame(player_info_italy).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_italy.csv', index=False)
pd.DataFrame(player_info_spain).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_spain.csv', index=False)
pd.DataFrame(player_info_france).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_france.csv', index=False)
pd.DataFrame(player_info_germany).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Dataframes/player_data_df_germany.csv', index=False)


# pd.DataFrame(df_39_columns_1).to_csv('/Users/vruiz.CDS/Downloads/Dataframes/consolidated_df.csv', index = False)